In [1]:
import pandas as pd
import numpy as np
import pymysql
import sqlalchemy
import re

In [2]:
filename = 'C:\\Users\\Paul\\Documents\\GitHub\\data-labs\\module-1\\16-pandas-project\\your-code\\GSAF5.csv\\GSAF5.csv'

In [3]:
df = pd.read_csv(filename, sep=';', encoding='latin1')

In [4]:
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-sept-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-sept-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-sept-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-sept-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-sept-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [5]:
#1_Identify missing values.
null_cols = df.isnull().sum()
null_cols[null_cols > 0]

Case Number                  9
Date                         2
Year                         3
Type                         1
Country                     44
Area                       402
Location                   498
Activity                   533
Name                       210
Sex                        581
Age                       2682
Injury                      51
Fatal (Y/N)                108
Time                      3227
Species                   2979
Investigator or Source     120
pdf                       1503
href formula              1503
href                      1506
Case Number.1             1503
Case Number.2             1503
original order            1503
Unnamed: 22               5981
Unnamed: 23               5981
dtype: int64

In [6]:
#2_Identify the shape of the dataframe to compare with missing values
df.shape

(5983, 24)

In [7]:
#3_Delete columns('Unnamed: 22', 'Unnamed: 23') with missing values with quantity pretty closed to the total number of rows 
drop_cols = list(null_cols[null_cols > 5980].index)
df = df.drop(drop_cols, axis=1)

In [8]:
#4_Delete duplicate columns and keep the one with the most values
df = df.drop(columns=['Case Number.1', 'Case Number.2'], axis = 1)
df = df.drop(columns=['href'], axis = 1)

In [9]:
#5_Replace missing values to '0'.
df = df.fillna('0')

In [10]:
#6_Look at the columns names
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula',
       'original order'],
      dtype='object')

In [11]:
#7_Remove all spaces by defining a fonction to delete them
def del_space(x):    
    if type(x) is str:        
        return x.strip()    
    return x
df = df.applymap(del_space)

In [12]:
#8_Rename incorrect columns
df.rename(columns = {'href formula':'Bibliography', 'original order':'Order', 'Sex ':'Sex', 'Species ':'Species'}, inplace = True)
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'Investigator or Source', 'pdf', 'Bibliography', 'Order'],
      dtype='object')

In [13]:
#9_Change order of columns
column_order = ['Order', 'Case Number', 'Date', 'Year', 'Time', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)',
       'Species', 'Investigator or Source', 'pdf', 'Bibliography']

df = df[column_order]
df

,Order,Case Number,Date,Year,Time,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Species,Investigator or Source,pdf,Bibliography
0,5993,2016.09.18.c,18-sept-16,2016,13h00,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,0,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
1,5992,2016.09.18.b,18-sept-16,2016,11h00,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,0,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
2,5991,2016.09.18.a,18-sept-16,2016,10h43,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,0,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
3,5990,2016.09.17,17-sept-16,2016,0,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,0,Struck by fin on chest & leg,N,0,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
4,5989,2016.09.15,16-sept-16,2016,0,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,0,No injury: Knocked off board by shark,N,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
5,5988,2016.09.15.R,15-sept-16,2016,0,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,0,0,Shark rammed boat. No injury to occupant,N,0,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
6,5987,2016.09.11,11-sept-16,2016,15h15,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,60s,Minor injury to arm,N,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
7,5986,2016.09.07,07-sept-16,2016,14h30,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,51,Severe lacerations to shoulder & forearm,N,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
8,5985,2016.09.06,06-sept-16,2016,15h40,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,50,FATAL,Y,0,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
9,5984,2016.09.05.b,05-sept-16,2016,Late afternoon,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,12,Lacerations & punctures to lower right leg,N,0,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...


In [14]:
#10_Remove "Reported" in column 'Date' and 'Occupant' in column 'Name'
# instead of coding => df = df.replace(('Reported', ''),('Occupant', ''))
df['Date'] = df['Date'].str.replace('Reported', '')
df['Name'] = df['Name'].str.replace('Occupant', '')

In [15]:
#11_Identify data types.
df.dtypes

Order                     object
Case Number               object
Date                      object
Year                      object
Time                      object
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Species                   object
Investigator or Source    object
pdf                       object
Bibliography              object
dtype: object

In [16]:
#12_look at the column 'Fatal Y/N'
df['Fatal (Y/N)'].value_counts(dropna = False)

N               4246
Y               1528
0                108
UNKNOWN           94
2014.10.11         1
n                  1
No injury          1
1931.03.15         1
#VALUE!            1
F                  1
2016.05.29.b       1
Name: Fatal (Y/N), dtype: int64

In [17]:
#12_Clean the values of this column
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].str.replace('#VALUE!','UNKNOWN')
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].str.replace('No injury','N')
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].str.replace('n','N')
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].str.replace('F','N')
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].str.replace('2014.10.11','UNKNOWN')
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].str.replace('1931.03.15', 'UNKNOWN')
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].str.replace('2016.05.29.b','UNKNOWN')
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].str.replace('\d','UNKNOWN')

print(set(df['Fatal (Y/N)']))

{'N', 'Y', 'UNKNOWN'}


In [18]:
#13_Count the column 'Species' values
df['Species'].value_counts(dropna = False)

0                                                                                                 2979
White shark                                                                                        156
Shark involvement not confirmed                                                                     81
Tiger shark                                                                                         66
Bull shark                                                                                          52
6' shark                                                                                            40
4' shark                                                                                            39
1.8 m [6'] shark                                                                                    36
1.5 m [5'] shark                                                                                    32
1.2 m [4'] shark                                                         

In [19]:
#14_Clean values of 'Species' column
df['Species'] = df['Species'].str.replace("6' shark","1.8 m [6'] shark")
df['Species'] = df['Species'].str.replace("5' shark","1.5 m [5'] shark")
df['Species'] = df['Species'].str.replace("4' to 1.5 m [5'] shark","1.5 m [5'] shark")
df['Species'] = df['Species'].str.replace("1.5 m shark","1.5 m [5'] shark")
df['Species'] = df['Species'].str.replace("4' shark","1.2 m [4'] shark")
df['Species'] = df['Species'].str.replace("3' to 1.2 m [4'] shark ","1.2 m [4'] shark")
df['Species'] = df['Species'].str.replace("3 m shark","3 m [10'] shark")
df['Species'] = df['Species'].str.replace("12' shark","3.7 m [12'] shark")

In [20]:
#15_Look and count the 'Time' column values to group by slot of the day
df['Time'].value_counts(dropna = False)

0                                                                        3227
Afternoon                                                                 172
11h00                                                                     122
12h00                                                                     106
Morning                                                                   106
15h00                                                                      99
16h00                                                                      93
14h00                                                                      93
16h30                                                                      72
13h00                                                                      71
14h30                                                                      70
17h30                                                                      68
17h00                                                           

In [21]:
# Based on the above value_counts results, 
# I identify where the data should be replaced to gather the maximum of relevant information to analyze

df['Time'] = df['Time'].str.replace('12h00', 'Noon') 
df['Time'] = df['Time'].str.replace('12h30', 'Noon') 
df['Time'] = df['Time'].str.replace('13h00', 'Noon') 
df['Time'] = df['Time'].str.replace('12h34', 'Noon') 
df['Time'] = df['Time'].str.replace('12h38', 'Noon') 
df['Time'] = df['Time'].str.replace('Midday.', 'Noon')
df['Time'] = df['Time'].str.replace('Midday', 'Noon')
df['Time'] = df['Time'].str.replace('Just after Noon', 'Noon')

df['Time'] = df['Time'].str.replace('"Evening"', 'Evening') 
df['Time'] = df['Time'].str.replace('Late Afternoon', 'Evening') 
df['Time'] = df['Time'].str.replace('"Early evening"', 'Evening') 
df['Time'] = df['Time'].str.replace('18h00', 'Evening')
df['Time'] = df['Time'].str.replace('18h30', 'Evening')

df['Time'] = df['Time'].str.replace('After midnight', 'Night') 
df['Time'] = df['Time'].str.replace('Shortly after midnight', 'Night') 
df['Time'] = df['Time'].str.replace('"Night"', 'Night') 
df['Time'] = df['Time'].str.replace('night', 'Night') 

df['Time'] = df['Time'].str.replace('After noon', 'Afternoon')
df['Time'] = df['Time'].str.replace('13h30', 'Afternoon')
df['Time'] = df['Time'].str.replace('13h45', 'Afternoon')
df['Time'] = df['Time'].str.replace('14h00', 'Afternoon')
df['Time'] = df['Time'].str.replace('14h30', 'Afternoon')                                    
df['Time'] = df['Time'].str.replace('15h00', 'Afternoon')
df['Time'] = df['Time'].str.replace('15h30', 'Afternoon')                                    
df['Time'] = df['Time'].str.replace('16h00', 'Afternoon')
df['Time'] = df['Time'].str.replace('16h30', 'Afternoon')                                    
df['Time'] = df['Time'].str.replace('17h00', 'Afternoon')
df['Time'] = df['Time'].str.replace('17h15', 'Afternoon')                                    
df['Time'] = df['Time'].str.replace('17h30', 'Afternoon')
df['Time'] = df['Time'].str.replace('P.M.', 'Afternoon')
                                    
df['Time'] = df['Time'].str.replace('Mid morning', 'Morning') 
df['Time'] = df['Time'].str.replace('Between 06h00 & 07h20', 'Morning') 
df['Time'] = df['Time'].str.replace('Early morning', 'Morning') 
df['Time'] = df['Time'].str.replace('Early Morning', 'Morning')
df['Time'] = df['Time'].str.replace('Morning / 11h30', 'Morning')
df['Time'] = df['Time'].str.replace('10h00 / Morning', 'Morning')
df['Time'] = df['Time'].str.replace('7h00', 'Morning')
df['Time'] = df['Time'].str.replace('7h30', 'Morning')
df['Time'] = df['Time'].str.replace('8h00', 'Morning')
df['Time'] = df['Time'].str.replace('8h30', 'Morning')
df['Time'] = df['Time'].str.replace('9h00', 'Morning')
df['Time'] = df['Time'].str.replace('9h30', 'Morning')
df['Time'] = df['Time'].str.replace('10h00', 'Morning')
df['Time'] = df['Time'].str.replace('10h30', 'Morning')
df['Time'] = df['Time'].str.replace('11h00', 'Morning') 
df['Time'] = df['Time'].str.replace('11h30', 'Morning')
df['Time'] = df['Time'].str.replace('1Morning', 'Morning')
df['Time'] = df['Time'].str.replace('0Morning', 'Morning')
df['Time'] = df['Time'].str.replace('A.M.', 'Morning')
df['Time'] = df['Time'].str.replace('Dusk', 'Morning')

In [22]:
# Export a 'clean' .csv version using Pandas
Sharks = df.to_csv('Sharks.csv', index=False)